In [1]:
#51:31

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
from pytesseract import pytesseract
from PIL import Image
import tempfile

In [2]:
pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
"""
One easy way, that was used by many OCR libs as well during many years is
to use the projected profile in 2D, if there is text, it will have some bumps, 
if not, it's an image.
It's very simple and efficient and doesn't require deep learning at all. 
"""

"\nOne easy way, that was used by many OCR libs as well during many years is\nto use the projected profile in 2D, if there is text, it will have some bumps, \nif not, it's an image.\nIt's very simple and efficient and doesn't require deep learning at all. \n"

In [4]:

def set_image_dpi(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    length_x, width_y,c = im.shape
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = cv2.resize(im, size, interpolation = cv2.INTER_AREA)
    return im

In [5]:
#Что нужно:
#Взять картинку и найти там хоть какой-то текст
#Выделить его
# Обрезать его полностью 
#Определить средний цвет текста
#Эроднуть выделенный цвет. Обрезать всё к хуям вне этих пикселей. (Это на цветных только)
#
#На черно белых - потыкаться с обрезаниями и т.д.
# Найти наибольшее скопление белого без полосок и тд.
#Там поискать текст. Если он там - выделяем и пытаемся его в текстовой формат перевести.

#https://github.com/AsiriAmalk/Comic_Speech_Bubble_Identifier_Using_Pytesseract
#https://github.com/harikc456/speech-bubble-extraction
#Это можно для баблов, потом нарезав их на баблы, уже можно искать в них текст.
#Для манхвы придётся извернуться, чтобы детектить там текст в них.

image_ball = set_image_dpi('../db/makimaText.png')
norm_img = np.zeros((image_ball.shape[0], image_ball.shape[1]))
image_balls = cv2.normalize(image_ball, norm_img, 0, 255, cv2.NORM_MINMAX)


image_balls = cv2.fastNlMeansDenoisingColored(image_balls, None, 10, 10, 7, 15)

kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(image_balls, kernel, iterations = 1)

image_balls = cv2.cvtColor(image_balls, cv2.COLOR_BGR2GRAY)

row, col = image_balls.shape[:2]
bottom = image_balls[row-2:row, 0:col]
mean = cv2.mean(bottom)[0]

border_size = 100
border = cv2.copyMakeBorder(
    image_balls,
    top=border_size,
    bottom=border_size,
    left=border_size,
    right=border_size,
    borderType=cv2.BORDER_CONSTANT,
    value=[255, 255, 255]
)

thresh = 0
im_bw = cv2.threshold(border, thresh, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

thresh = cv2.adaptiveThreshold(
        im_bw, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

kernel = np.ones((3, 3), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)
eroded = cv2.erode(dilated, kernel, iterations=1)

magem = cv2.bitwise_not(im_bw)

letter_boxes = pytesseract.image_to_boxes(image_ball,lang='eng')

height, width,c = image_ball.shape

for box in letter_boxes.splitlines():
    box = box.split()
    x,y,w,h = int(box[1]),int(box[2]),int(box[3]),int(box[4])
    cv2.rectangle(image_ball, (x, height - y), (w, height - h), (255,0,0), 3)
    cv2.putText(image_ball, box[0], (x,height-h+32), cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
    print(box)
    
cv2.imshow("window", image_ball)
cv2.waitKey(0)

['G', '164', '157', '232', '192', '0']
['Z', '207', '157', '232', '192', '0']
['A', '103', '149', '168', '173', '0']
['N', '190', '161', '196', '166', '0']
['=', '92', '129', '136', '157', '0']
['~', '62', '0', '1032', '707', '0']


-1